In [28]:
from tqdm import tqdm_notebook

from data_reader import DataReader
from collections import defaultdict
import numpy as np

In [70]:
class IBM2:
    def __init__(self,longest_s, source_path: str, target_path) -> None:
        self.data_reader = DataReader(source_path, target_path)

        init_ef = 1 / (self.data_reader.n_source_tokens * self.data_reader.n_target_tokens)
        
        init_align = 1 / (longest_s)
        
        self.probs_ef: Dict[Tuple[str, str], float] = defaultdict(lambda: init_ef)
        
        self.align_probs : Dict[int, float] = defaultdict(lambda: init_align)

    def jump_func(self,e_pos,f_pos,len_e,len_f):
                    
            
        return int(e_pos - np.floor(f_pos * (len_e/len_f)))
    
        
    def train(self, n_iter: int):
        for s in range(n_iter):
            
            counts_ef = defaultdict(float)
            counts_e = defaultdict(float)
            counts_align = defaultdict(int)
            for k in tqdm_notebook(range(len(self.data_reader))):
                e, f = self.data_reader[k]
                f = [NULL_TOKEN] + f

                e_normalizer = defaultdict(float)
                
                for e_pos,we in enumerate(e):
                    for f_pos,wf in enumerate(f):
                        x = self.jump_func(e_pos,f_pos,len(e),len(f))
                        align_prob = self.align_probs[x]
                        
                        e_normalizer[we] += self.probs_ef[we, wf] * align_prob

                for e_pos,we in enumerate(e):
                    for f_pos,wf in enumerate(f):
                        x = self.jump_func(e_pos,f_pos,len(e),len(f))
                        align_prob = self.align_probs[x]
                        
                        delta = (self.probs_ef[we, wf] * align_prob) / e_normalizer[we]

                        counts_ef[we, wf] += delta
                        
                        counts_e[wf] += delta
                        
                        counts_align[x] += delta
                        
                        

            for (we, wf), c in counts_ef.items():
                self.probs_ef[we, wf] = c / counts_e[wf]
            
            norm_align_probs = np.sum(list(counts_align.values()))
            
            for x,c in counts_align.items():
            
                self.align_probs[x] = c/norm_align_probs

In [71]:
ibm2 = IBM2(80,'training/hansards.36.2.e', 'training/hansards.36.2.f')
ibm2.train(1)

KeyboardInterrupt: 